In [ ]:
# cell 1: 导入库
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO
import base64

In [ ]:
# cell 2: 邮票类
class Stamp:
    def __init__(self, value, color, name):
        self.value = value
        self.color = color
        self.name = name
    
    def create_image(self):
        fig, ax = plt.subplots(figsize=(2, 2))
        ax.add_patch(plt.Rectangle((0.1, 0.1), 0.8, 0.8, 
                                 facecolor=self.color, 
                                 edgecolor='white', 
                                 linewidth=4))
        ax.text(0.5, 0.5, str(self.value), 
               fontsize=24, ha='center', va='center', 
               color='white', weight='bold')
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')
        
        # 转换为 base64
        buf = BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight', dpi=80)
        plt.close(fig)
        buf.seek(0)
        img_str = base64.b64encode(buf.read()).decode()
        return f"data:image/png;base64,{img_str}"
    
    def create_button(self):
        """创建带有图像的按钮"""
        # 创建图像和标签的HTML
        image_html = f'<img src="{self.create_image()}" width="60" height="60"><br>{self.name}'
        
        # 使用HTML小部件显示图像
        image_widget = widgets.HTML(value=image_html)
        
        # 创建按钮
        button = widgets.Button(
            description='',
            layout=widgets.Layout(width='80px', height='100px')
        )
        
        # 将图像和按钮组合在一起
        container = widgets.VBox([image_widget, button])
        
        return container, button

In [ ]:
# cell 3: 游戏状态管理
class StampGameState:
    def __init__(self):
        self.work_area = {
            'unit': 0,    # 个位
            'ten': 0,     # 十位
            'hundred': 0, # 百位
            'thousand': 0 # 千位
        }
        self.total_value = 0
        self.feedback = "欢迎使用数学邮票游戏！"
        
        # 定义邮票类型
        self.stamp_types = {
            'unit': Stamp(1, '#2E8B57', '个位'),
            'ten': Stamp(10, '#4169E1', '十位'),
            'hundred': Stamp(100, '#DC143C', '百位'),
            'thousand': Stamp(1000, '#8A2BE2', '千位')
        }

In [ ]:
# cell 4: 创建界面控件
def create_stamp_game():
    state = StampGameState()
    
    # 创建邮票按钮
    stamp_containers = {}
    stamp_buttons = {}
    
    for stamp_type, stamp in state.stamp_types.items():
        container, button = stamp.create_button()
        button.stamp_type = stamp_type
        stamp_containers[stamp_type] = container
        stamp_buttons[stamp_type] = button
    
    # 工作区显示
    work_area_display = widgets.HTML()
    
    # 控制按钮
    clear_button = widgets.Button(description="🗑️ 清空", layout=widgets.Layout(width='100px'))
    calculate_button = widgets.Button(description="🧮 计算", layout=widgets.Layout(width='100px'))
    auto_exchange = widgets.Checkbox(value=True, description='自动进位')
    
    # 反馈显示
    feedback_display = widgets.HTML()
    
    # 更新显示函数
    def update_display():
        # 更新工作区显示
        work_html = "<div style='background: #f0f8ff; padding: 15px; border-radius: 10px;'>"
        work_html += "<h3>📦 工作区</h3>"
        
        for stamp_type, count in state.work_area.items():
            if count > 0:
                stamp = state.stamp_types[stamp_type]
                work_html += f"""
                <div style='display: inline-block; text-align: center; margin: 10px;'>
                    <img src="{stamp.create_image()}" width="50" height="50"><br>
                    <span style='font-weight: bold;'>× {count}</span>
                </div>
                """
        
        work_html += f"<div style='margin-top: 15px; font-size: 18px;'><strong>总值: {state.total_value}</strong></div>"
        work_html += "</div>"
        work_area_display.value = work_html
        
        # 更新反馈
        feedback_display.value = f"""
        <div style='background: #fffacd; padding: 10px; border-radius: 5px; margin: 10px 0;'>
            {state.feedback}
        </div>
        """
    
    # 按钮事件处理
    def on_stamp_click(btn):
        stamp_type = btn.stamp_type
        state.work_area[stamp_type] += 1
        
        # 自动进位
        if auto_exchange.value:
            auto_exchange_stamps()
        
        calculate_total()
        update_display()
    
    def auto_exchange_stamps():
        # 10个个位 -> 1个十位
        if state.work_area['unit'] >= 10:
            state.work_area['unit'] -= 10
            state.work_area['ten'] += 1
            state.feedback = "🔁 自动进位：10个个位 → 1个十位"
        
        # 10个十位 -> 1个百位
        if state.work_area['ten'] >= 10:
            state.work_area['ten'] -= 10
            state.work_area['hundred'] += 1
            state.feedback = "🔁 自动进位：10个十位 → 1个百位"
        
        # 10个百位 -> 1个千位
        if state.work_area['hundred'] >= 10:
            state.work_area['hundred'] -= 10
            state.work_area['thousand'] += 1
            state.feedback = "🔁 自动进位：10个百位 → 1个千位"
    
    def calculate_total():
        state.total_value = (
            state.work_area['unit'] * 1 +
            state.work_area['ten'] * 10 +
            state.work_area['hundred'] * 100 +
            state.work_area['thousand'] * 1000
        )
    
    def on_clear_click(btn):
        for key in state.work_area:
            state.work_area[key] = 0
        state.total_value = 0
        state.feedback = "工作区已清空"
        update_display()
    
    def on_calculate_click(btn):
        calculate_total()
        state.feedback = f"计算结果：{state.total_value}"
        update_display()
    
    # 绑定事件
    for button in stamp_buttons.values():
        button.on_click(on_stamp_click)
    
    clear_button.on_click(on_clear_click)
    calculate_button.on_click(on_calculate_click)
    
    # 初始显示
    update_display()
    
    # 布局
    stamp_buttons_row = widgets.HBox(list(stamp_containers.values()))
    control_row = widgets.HBox([clear_button, calculate_button, auto_exchange])
    
    # 标题和说明
    title = widgets.HTML("""
    <div style='text-align: center; background: linear-gradient(45deg, #4CAF50, #2196F3); 
                padding: 20px; border-radius: 10px; color: white; margin: 10px 0;'>
        <h1>🎴 数学邮票游戏</h1>
        <p>点击邮票添加到工作区，体验数学的乐趣！</p>
    </div>
    """)
    
    # 组装界面
    app = widgets.VBox([
        title,
        widgets.HTML("<h2>🎯 选择邮票</h2>"),
        stamp_buttons_row,
        widgets.HTML("<h2>📊 工作区</h2>"),
        work_area_display,
        widgets.HTML("<h2>⚙️ 控制面板</h2>"),
        control_row,
        widgets.HTML("<h2>💬 反馈</h2>"),
        feedback_display
    ])
    
    return app

In [ ]:
# cell 5: 启动游戏
game_app = create_stamp_game()
display(game_app)